In [1]:
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

from utils import GLOBAL, functions

In [2]:
import pandas as pd

In [3]:
proj_df = pd.read_csv(GLOBAL.PROJECTIONS_2021, index_col=0)

In [5]:
### DO NOT RUN IF USING 2021 PROJECTIONS OR HIGHER ####
### Section is for older projections newer ones have the scoring columns ###

#Get rid of odd column if there are any[row indexer, column indexer], change 0 to 1 if there are
proj_df = proj_df.iloc[:, 0:]

#Add in Fantasy Points based on scoring type
proj_df['FantasyPoints'] = (
    functions.scoringCalculator(
        proj_df['Receptions'], proj_df['ReceivingYds'], proj_df['ReceivingTD'], proj_df['FL'], proj_df['RushingYds'], proj_df['RushingTD'], proj_df['PassingYds'], proj_df['PassingTD'], proj_df['Int'], GLOBAL.HALF_PPR_SCORING
    )
)

#Get dataframes of specific pos
rb_proj_df = proj_df[proj_df['Pos'] == 'RB', ['Player, Team, Pos, Receptions, ReceivingYds, FL, ReceivingTD, RushingAtt, RushingYds, RushingTD, PassingAtt, PassingYds, PassingTD, Int']]
wr_proj_df = proj_df[proj_df['Pos'] == 'WR', ['Player, Team, Pos, Receptions, ReceivingYds, FL, ReceivingTD, RushingAtt, RushingYds, RushingTD, PassingAtt, PassingYds, PassingTD, Int']]
te_proj_df = proj_df[proj_df['Pos'] == 'TE', ['Player, Team, Pos, Receptions, ReceivingYds, FL, ReceivingTD, RushingAtt, RushingYds, RushingTD, PassingAtt, PassingYds, PassingTD, Int']]
qb_proj_df = proj_df[proj_df['Pos'] == 'QB', ['Player, Team, Pos, Receptions, ReceivingYds, FL, ReceivingTD, RushingAtt, RushingYds, RushingTD, PassingAtt, PassingYds, PassingTD, Int']]

In [4]:
#Get dataframes of specific pos
rb_proj_df = proj_df[proj_df['Pos'] == 'RB']
wr_proj_df = proj_df[proj_df['Pos'] == 'WR']
te_proj_df = proj_df[proj_df['Pos'] == 'TE']
qb_proj_df = proj_df[proj_df['Pos'] == 'QB']

In [44]:
#ADP for current year and scoring format
adp_df = pd.read_csv(GLOBAL.ADP_2021_HALF_PPR, index_col=0)

adp_df['ADP RANK'] = adp_df['AVG'].rank()

adp_df_cutoff = adp_df[:100]

replacement_players = {
    'RB': '',
    'WR': '',
    'TE': '',
    'QB': ''
}

In [46]:
#ONLY FOR OLDER DATASETS
for _, row in adp_df_cutoff.iterrows():
    position = row['POS'][:1]
    print(position)
    player = row['Player']

    if position in replacement_players:
        replacement_players[position] = player

In [40]:
replacement_players

{'RB': '', 'WR': '', 'TE': '', 'QB': ''}

In [38]:
adp_df

,Player,Team,Bye,POS,Yahoo,Fantrax,FFC,AVG,ADP RANK
Rank,,,,,,,,,
1.0,Christian McCaffrey,CAR,13.0,RB1,1.0,1.0,1.0,1.0,1.0
2.0,Dalvin Cook,MIN,7.0,RB2,2.0,2.0,2.0,2.0,2.0
3.0,Derrick Henry,TEN,13.0,RB3,3.0,3.0,3.0,3.0,3.0
4.0,Alvin Kamara,NO,6.0,RB4,4.0,4.0,4.0,4.0,4.0
5.0,Saquon Barkley,NYG,10.0,RB5,6.0,5.0,6.0,5.7,5.0
...,...,...,...,...,...,...,...,...,...
390.0,N'Keal Harry,NE,14.0,WR136,NaN,389.0,NaN,389.0,385.0
391.0,Damiere Byrd,CHI,10.0,WR137,NaN,390.0,NaN,390.0,386.0
392.0,Devin Smith,NE,14.0,WR138,NaN,391.0,NaN,391.0,387.0
